In [10]:
### Пояснялки в комментариях
from matplotlib import pyplot as plt
### Пишешь не в jupyter - комментируй эту строку
%config InlineBackend.figure_formats = ['svg']
### Эту тоже
from ipywidgets import interact
### Это самая важная в мире библиотека без нее никуда
import numpy as np
import scipy as sp
from pylab import *
from IPython.display import clear_output, HTML, display, Image

In [11]:
# Задание параметров
Lx = Ly = 1.0; Lz=0.1;
T = 0.1
Nx = 50
Ny = 50
Nz = 50
Nt = 100
dx = Lx / Nx
dy = Ly / Ny
dz = Lz / Nz
dt = T / Nt
print(dx+dy+dt+dz)
Dxx = 1/dx**2; Dyy = 1/dy**2; Dzz=1/dz**2;
X = np.linspace(0, Lx, Nx);
Y = np.linspace(0, Ly, Ny);
Z = np.linspace(0, Lz, Nz);
Tset = np.linspace(0, T, Nt);
a = 0.05

0.043000000000000003


In [12]:
# Инициализация начальных условий
# Sorce of heat
def f(x, y, z, t):
    if (np.abs(x)>0.7 and np.abs(y)>0.7 and np.abs(z)<0.3): 
        return 5+t
    return 0

def F(TT, dx, dy, dz):
    F_vals = np.zeros((Nt, Nx, Ny, Nz))
    for i in range (1, Nx - 1):
        for j in range(1, Ny-1):
            for k in range(1, Nz-1):
                xi = i*dx; yj = j*dy; zk = k*dz
                F_vals[:, i, j, k] = f(xi, yj, zk, Tset)

    return F_vals
F_vals = F(Tset, dx, dy, dz)
# Initial conditions
u0 = np.zeros((Nx, Ny, Nz))
for i in range(1, Nx-1):
    for j in range(1, Ny-1):
        u0[i, j, 0:-20] = np.exp(-((X[i]-0.3)**2 + (Y[j]-0.3)**2)/0.1)

In [59]:
# Создание интерактивного интерфейса
def displayZT(n, k, X, Y, U, ax):
    #n = int(round(n/100*Nz, 0))
    if n>Nz-1:
        n = Nz-1
    #k = int(round(k/100*Nt, 0))
    if k>Nt-1:
        k = Nt-1
    cp1=ax.contourf(X,Y,U[k, :, :, n],10,cmap='PuOr')
    #plt.colorbar(cp1)
    cp1=ax.contour(X,Y,U[k, :, :, n],10,colors='k')
    plt.clabel(cp1,inline=True, fontsize=10)

In [19]:
# Explicit
# Функция, реализующая явную разностную схему для уравнения теплопроводности
def solve_layer_exp(u, a, f, dt, dx, dy, dz):
    unew = u.copy()
    Nx, Ny, Nz = u.shape
    for k in range(1, Nz - 1):
        for j in range(1, Ny - 1):
            for i in range(1, Nx - 1):
                unew[i, j, k] = u[i, j, k] + dt * (
                    a * (
                        (u[i+1, j, k] - 2*u[i, j, k] + u[i-1, j, k]) * Dxx + 
                        (u[i, j+1, k] - 2*u[i, j, k] + u[i, j-1, k]) * Dyy + 
                        (u[i, j, k+1] - 2*u[i, j, k] + u[i, j, k-1]) * Dyy
                    ) + f[i, j, k]
                )

    # Update Newman conditions
    unew[0, :, :] = unew[1, :, :];
    unew[-1, :, :] = unew[-2, :, :];
    unew[:, 0, :] = unew[:, 1, :];
    unew[:, -1, :] = unew[:, -2, :];
    unew[:, :, 0] = unew[:, :, 1];
    unew[:, :, -1] = unew[:, :, -2];
    return unew

def solve_exp():
    U = np.zeros((Nt, Nx, Ny, Nz))
    #U[:, 0, :, :] = U[:, -1, :, :] = U[:, :, 0, :] = U[:, :, -1, :] = U[:, :, :, 0] = U[:, :, :, -1]
    u = np.copy(u0)
    U[0] = u0
    for n in range(1, Nt):
        clear_output(wait = True)
        print(f'Progress: {round(n/(Nt), 2)*100}%')  
        u = solve_layer_exp(u, a, F_vals[n], dt, dx, dy, dz)
        U[n] = u
    return U

In [45]:
# Implicit

def matrix(a, b):
    N = len(X)
    main_diag = np.eye(N)
    help_diag = np.eye(N-1)
    main_diag = main_diag*b
    help_diag = -help_diag*a
    main_diag[1:, :-1] += help_diag
    main_diag[:-1, 1:] += help_diag
    main_diag[0, 0] = 1
    main_diag[0, 1] = -1
    main_diag[-1, -1] = -1
    main_diag[-1, -2] = 1
    return main_diag

def solve_layer_imp(u0, nt, f):
    u1_3 = np.zeros((Nx, Ny, Nz)) 
    # layer X
    an = a*dt/(1*dx**2)
    bn = 1 + 2*a*dt*Dxx/(1)
    Ax = matrix(an, bn)
    #print(A)
    Bx = np.zeros(Nx)
    for k in range(Nz):
        for j in range(Ny): 
            Bx[0] = Bx[-1] = 0
            Bx[1:-1] = u0[1:-1, j, k] + dt*f[nt, 1:-1, j, k]/3
            u1_3[:, j, k] = sp.linalg.solve(Ax, Bx)
    #layer Y
    u2_3 = np.zeros((Nx, Ny, Nz)) 
    an = a*dt/(1*dy**2)
    bn = 1 + 2*a*dt*Dyy/(1)
    Ay = matrix(an, bn)
    By = np.zeros(Ny)
    for k in range(Nz):
        for i in range(Nx):
            By[0] = By[-1] = 0
            By[1:-1] = u1_3[i, 1:-1, k] + dt*f[nt, i, 1:-1, k]/3
            u2_3[i, :, k] = sp.linalg.solve(Ay, By)

    #layer Z
    u = np.zeros((Nx, Ny, Nz)) 
    an = a*dt*Dzz/(1)
    bn = 1 + 2*a*dt/(1*dz**2)
    Az = matrix(an, bn)
    Bz = np.zeros(Nz)
    for j in range(Ny):
        for i in range(Nx):
            By[0] = By[-1] = 0
            By[1:-1] = u2_3[i, j, 1:-1] + dt*f[nt, i, j, 1:-1]/3
            u[i, j, :] = sp.linalg.solve(Ay, By)    
    return u

def solve_imp(u0, f):
    U = np.zeros((Nt, Nx, Ny, Nz))
    U[0] = u0
    for n in range(1, Nt):
        clear_output(wait = True)
        print(f'Progress: {round(n/(Nt), 2)*100}%') 
        U[n] = solve_layer_imp(U[n-1], n, F_vals)
    return U

In [42]:
U_exp = solve_exp()

Progress: 99.0%


In [46]:
U_imp = solve_imp(u0, F_vals)

Progress: 99.0%


In [47]:
def var(n, k):
    fig, (ax, ay) = plt.subplots(1, 2, figsize=(10, 6))
    ax.set_title("explicit")
    ay.set_title("implicit")
    fig.suptitle(f"t={Nt*(k/Nt)*dt}\nz={Nz*(n/Nz)*dz}")
    displayZT(n, k, X, Y, U_exp, ax)
    displayZT(n, k, X, Y, U_imp, ay)
    show()
interact(var, n=(0, Nz, 1), k=(0, Nt, 1))

interactive(children=(IntSlider(value=25, description='n', max=50), IntSlider(value=50, description='k'), Outp…

<function __main__.var(n, k)>

In [60]:
DU = np.abs(U_imp-U_exp)
def var(n, k):
    fig, ax = plt.subplots(figsize=(10, 6))
    fig.suptitle(r"$|U_{exp}-U_{imp}|$")
    fig.suptitle(r"$|\delta U|$"+f"(t={Nt*(k/Nt)*dt}, z={Nz*(n/Nz)*dz})")
    displayZT(n, k, X, Y, DU, ax)
    show()
interact(var, n=(0, Nz, 1), k=(0, Nt, 1))

interactive(children=(IntSlider(value=25, description='n', max=50), IntSlider(value=50, description='k'), Outp…

<function __main__.var(n, k)>